# Mappeoppgave 1 - Samfunnsøkonomisk analyse av en turistskatt

### Kandidat Number: --

In [13]:
# SETUP

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sympy as sp
from sympy.solvers import solve

from cycler import cycler
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as mticker
import matplotlib.pylab as pylab

from pyjstat import pyjstat
import requests

from IPython.display import display, HTML

# custome plot style
params  = {
"lines.linewidth": 1.5,

"legend.fancybox": "true",

"axes.prop_cycle": cycler('color', ["#ffa822","#1ac0c6","#ff6150","#30B66A","#B06AFF","#FF21E1"]),
"axes.facecolor": "#2b2b2b",
"axes.axisbelow": "true",
"axes.grid": "true",
"axes.edgecolor": "#2b2b2b",
"axes.linewidth": 0.5,
"axes.labelpad": 0,

"patch.edgecolor": "#2b2b2b",
"patch.linewidth": 0.5,

"grid.linestyle": "--",
"grid.linewidth": 0.5,
"grid.color": "#b8aba7",

"xtick.major.size": 0,
"xtick.minor.size": 0,
"ytick.major.size": 0,
"ytick.minor.size": 0,

"font.family":"monospace",
"font.size":10.0,
"text.color": "#FFE9E3",
"axes.labelcolor": "#b8aba7",
"xtick.color": "#b8aba7",
"ytick.color": "#b8aba7",

"savefig.edgecolor": "#2b2b2b",
"savefig.facecolor": "#2b2b2b",

"figure.subplot.left": 0.08,
"figure.subplot.right": 0.95,
"figure.subplot.bottom": 0.09,
"figure.facecolor": "#2b2b2b"}

pylab.rcParams.update(params)
#print("finish")

In [15]:
POST_URL = "https://data.ssb.no/api/v0/no/table/10638/"

payload = {
  "query": [
    {
      "code": "KonsumInnd",
      "selection": {
        "filter": "item",
        "values": [
          "TOTK",
          "UTLK",
          "NOHT",
          "NONF"
        ]
      }
    },
    {
      "code": "Reiseliv",
      "selection": {
        "filter": "item",
        "values": [
          "0",
          "01",
          "01.01",
          "01.02",
          "01.03",
          "01.04",
          "01.05",
          "01.06",
          "01.07",
          "01.08",
          "01.09",
          "01.10",
          "02"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2011",
          "2012",
          "2013",
          "2014",
          "2015",
          "2016",
          "2017",
          "2018",
          "2019",
          "2020",
          "2021"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

resultat = requests.post(POST_URL, json = payload)
dataset = pyjstat.Dataset.read(resultat.text)
df = dataset.write('dataframe')

POST_URL = "https://data.ssb.no/api/v0/no/table/10603/"

payload = {
  "query": [
    {
      "code": "Reiseliv",
      "selection": {
        "filter": "item",
        "values": [
          "NORTOT",
          "OTV",
          "SEV",
          "TRJ",
          "TRBSD",
          "TRSF",
          "TRF",
          "UTL",
          "RBAV",
          "KUV",
          "SAF"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

resultat = requests.post(POST_URL, json = payload)
dataset = pyjstat.Dataset.read(resultat.text)
df2 = dataset.write('dataframe')

In [18]:
#display(df)
#display(df2)

,konsumentgruppe,reiselivsnæring,statistikkvariabel,år,value
0,Totalt konsum,Turistkonsum i alt,Løpende priser (mill. kr),2011,128526.0
1,Totalt konsum,Turistkonsum i alt,Løpende priser (mill. kr),2012,136882.0
2,Totalt konsum,Turistkonsum i alt,Løpende priser (mill. kr),2013,145156.0
3,Totalt konsum,Turistkonsum i alt,Løpende priser (mill. kr),2014,151416.0
4,Totalt konsum,Turistkonsum i alt,Løpende priser (mill. kr),2015,158669.0
...,...,...,...,...,...
1139,Norske næringers utgifter til forretningsreise...,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2017,0.0
1140,Norske næringers utgifter til forretningsreise...,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2018,0.0
1141,Norske næringers utgifter til forretningsreise...,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2019,0.0
1142,Norske næringers utgifter til forretningsreise...,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2020,0.0


,hovedstørrelse,reiselivsnæring,statistikkvariabel,år,value
0,"Produksjon, basisverdi",SUM REISELIVSNÆRINGENE,Løpende priser,2011,221829.0
1,"Produksjon, basisverdi",SUM REISELIVSNÆRINGENE,Løpende priser,2012,233136.0
2,"Produksjon, basisverdi",SUM REISELIVSNÆRINGENE,Løpende priser,2013,247140.0
3,"Produksjon, basisverdi",SUM REISELIVSNÆRINGENE,Løpende priser,2014,262754.0
4,"Produksjon, basisverdi",SUM REISELIVSNÆRINGENE,Løpende priser,2015,273318.0
...,...,...,...,...,...
721,Bruttoinvestering i fast realkapital,Sport og annen fritidsvirksomhet,Faste 2020-priser,2017,1527.0
722,Bruttoinvestering i fast realkapital,Sport og annen fritidsvirksomhet,Faste 2020-priser,2018,1777.0
723,Bruttoinvestering i fast realkapital,Sport og annen fritidsvirksomhet,Faste 2020-priser,2019,4180.0
724,Bruttoinvestering i fast realkapital,Sport og annen fritidsvirksomhet,Faste 2020-priser,2020,1968.0


### Kapittel 1 - Innledning

Når du skal på ferie, så er det flere ting som kan være med på at det stedet du velger blir valgt. Om det er naturen, strendene og havet, heller kulturen og historien, slik områder ofte også har museumer får historien, guided turer slik som hval sightisng i tromsø, heller natur områder slik som parker heller strender. Uansett ka du velger så må de tingene velikesholdes og for det så trenger man penger.

Og det er der turistskatt kommer inn. Tanken er at med å skatte turister med en lokal skatt som kan brukes av staten heller turist organisasjoner så kan intekten fra skatten bli brukt får å velikeholding og mulig utbygging av turist atraksjoner, så i teorien skal det ha en posetive effect på turistmarkedet, flere og bedre atraksjoner betyr flere turister, dermed kan lokal bedriftene tjene mer.

Turistskatt er ofte lagt på toppen av overnatts områder slik som hoteler heller airbnb. Det kan også bli lagt til atraksjoner slik som museumer heller guided turer. 

Så du kan forestil at du bestemer deg å dra på ferie til Tromsø, og når du skjekker inn på hotelet så betaler du en turistskatt når du skjekker inn, og dagen etterpå karnsje du tar deg en guided tur får å se nordlys og der kan det være at du også må betale du en litten extra skatt.

#### Hvor popelert er turistskatt og hvordan gikk det der

#### Posetive og Negative

#### Hvordan vil vi se paa d

### Kapittel 2 - Turistnæringen i Norge

Explane the marked (Choose a marked to foucus on)
Se om vi kan spesifikk se inn paa noen omraader slik som Lofoten siden oppgaven neveder d

Show at spesific data, and show graphs. First general and the spesific

Note fokuser paa hotel og overnatting, men skjekk om eg kan inne nokka fly data faar aa teste fly transport mot trafik.


### Kapittel 3 - Samfunnsøkonomiske effekter av en turistskatt

Analyze the marked, some examples:

* Hva er formålet med å innføre en turistskatt på overnattinger?
* Hvordan fungerer markedet for overnattinger uten en skatt?
* Hvilke endringer forventer vi etter at skatten blir innført?
* Hvilke effektivitets- og fordelingseffekter har dette tiltaket?
* Hvor store effekter forventer vi, og hva er dette avhengig av?
* Finnes det noen estimat på sentrale parameter (for eksempel priselastisiteten på etterspørsel etter overnattinger) som vi kan bruke for å konkretisere analysen?
* En vurdering av hvorvidt formålet med tiltaket oppfylles.

I kurset så langt har vi brukt ulike varianter av markedskrysset for å analysere forskjellige markeder, og dette blir verktøyet som brukes her også. Husk å gi en god forklaring av alle figurer som du lager i forbindelse med analysen. Dere har alt dere trenger! (Om du vil gå noe utover det som vi har snakket om i kurset så langt kan det være en idé å se på kapittel 13 i læreboka om "eksterne virkninger").

### Kapittel 4 - Konklusjon


Her oppsummerer du kort dinne funn fra analysen, og råd til Nærings- og fiskeridepartementet.